In [ ]:
import numpy as np
import math as math
import csv as csv
import sys

In [ ]:
def entropy(pred_list):
        value,label_quantity = np.unique(pred_list, return_counts=True)
        length = len(pred_list)
        if length == 0:
            return 0
        else:
            prob  = label_quantity/length
            en = 0 
            for c in prob:
                en += c*m.log2(c)
            return (-en)
    


In [ ]:
def mutual_information(curr_var, ds):
        #calculating conditional entropy 
        predict_list = []
        var_list = []
        for i in range(len(ds)):
            predict_list.append(ds[i][-1]) 
            var_list.append(ds[i][curr_var])
        var_el = list(set(var_list))
        final_list = list(zip(var_list,predict_list))
        split1 = []
        split2 = []
        for tup in (final_list):
            if tup[0]== var_el[0]:
                split1.append(tup[-1])
            elif tup [0]== var_el[1]:
                split2.append(tup[-1])
        total_len= len(split1)+len(split2)
        cond_en = (len(split1)/total_len)*entropy(split1) + (len(split2)/total_len)*entropy(split2)

In [ ]:
def get_best_index_node (ds):
        variables = ds[0][:-1]
        mi = []
        for v in range(len(variables)):
            mi.append(mutual_information(v,ds))
        index = mi.index(max(mi))
        return {"index":index}


In [ ]:
def get_split (index, data_list):
    #print("index:",index)
        select = []
        for i in data_list:
            select.append(i[index])
        select=list(set(select))
        list_1=[]
        list_2=[]
        for i in data_list:
            if  i[index]==select[0]:
                list_1.append(i)
            elif i[index]==select[1]:
                list_2.append(i)
        a = len(list_1) + len(list_2)
        return [select,[list_1,list_2]]


In [ ]:
def TrainStump (node, ds, max_depth, depth):
    
   
        if len(ds) <=1:
            node['predict']=classify(ds)
            return
    
        splits = get_split(node["index"],ds)
        num_att = len(ds[0])-1
    
        
    
        if mutual_information(node["index"],ds) <= 0.0 or mutual_information(node["index"],ds) == -0.0 :
            print(node)
            node['predict'] = classify(splits[1][0]+splits[1][1])
        
        
            return

        if depth >= max_depth: 
            print(node)
            node['predict']= classify(splits[1][0]+splits[1][1])
        
            return
   
        if depth >= num_att: 
            print(node)
            node['predict']= classify(splits[1][0]+splits[1][1])
        
            return
    
        else:  
        
        
            if len(splits[1][0])==0:
                node["left"]=classify(splits[1][0]+splits[1][1])
                node["right"]=get_best_index_node (splits[1][1])
                node["left"]["val"]=splits[0][0]
                node["right"]["val"]=splits[0][1]
                TrainStump(node["right"],splits[1][1],max_depth, depth+1)
            
            
        
            if len(splits[1][1])==0:
           
                node["right"]=classify(splits[1][0]+splits[1][1])
                node["left"]=get_best_index_node (splits[1][1])
            
                node["left"]["val"]=splits[0][0]
                node["right"]["val"]=splits[0][1]
                TrainStump(node["left"],splits[1][0],max_depth, depth+1)
        
            else:
                node["left"]=get_best_index_node (splits[1][0])
                node["right"]=get_best_index_node (splits[1][1])
    
                node["left"]["val"]=splits[0][0]
                node["right"]["val"]=splits[0][1]
                TrainStump(node["left"],splits[1][0],max_depth, depth+1)
                TrainStump(node["right"],splits[1][1],max_depth, depth+1)
            
        

In [ ]:
def CreateTree (train_data, max_depth):
        root = get_best_index_node(train_data)
        TrainStump(root,train_data,max_depth, depth=0)
        return root

In [ ]:
def print_tree(node, depth=0):
        if isinstance(node, dict):
            print(depth*' ',depth, 'f'+str(node['index']))
            if "left" in node.keys():
                print_tree(node['left'], depth+1)
            if "right" in node.keys():
                print_tree(node['right'], depth+1)
        else:
            print(depth*' ',depth, node)

In [ ]:
def classify (split):
        class_data=[]
        for r in split:
            class_data.append(r[-1])

        return max(class_data, key=class_data.count)


In [ ]:
def predict (test_data, tree):
        if "predict" in tree.keys():
            return (tree["predict"])
        
    
        else:
            if test_data[tree["index"]]==tree["left"]["val"]:
                return(predict(test_data, tree["left"]))
        
            if test_data[tree["index"]]==tree["right"]["val"]:
                return(predict(test_data, tree["right"]))
        
        

In [ ]:
def error_rate (dataset, tree):
        correct=[]
        wrong=[]
        for row in dataset:
            p=predict(row,tree)
            if p == row[-1]:
                correct.append(p)
            else:
                wrong.append(p)
        error_rate = 1-(len(correct)/(len(correct)+len(wrong)))
        return error_rate
        

In [ ]:
if __name__ == '__main__':
    with open (sys.argv[1],"rt") as input1:
            list1 = []
            for line in input1:
                line = line[:-1].split(',')
                list1.append(line)
            list1.pop(0)
            dataset1=list1
            tree=CreateTree(dataset1,int(sys.argv[3]))




    with open (sys.argv[4],"wt") as output1:
        for row in dataset1:
            a=predict(row,tree)
            output1.write(a +'\n')
            
    
    with open (sys.argv[2],"rt") as input2:
        list2 = []
        for line in input2:
            line = line[:-1].split(',')
            list2.append(line)
        list2.pop(0)
        dataset2=list2
        



    with open (sys.argv[5],"wt") as output2:
        for row in dataset2:
            a=predict(row,tree)
            output2.write(a +'\n')
        

    with open (sys.argv[6],"wt") as output3:
        err_rate_train = error_rate (dataset1,tree)
        err_rate_test = error_rate (dataset2,tree)
        output3.write("error(train): " + str(err_rate_train)+"\n")
        output3.write("error(test): " + str(err_rate_test))
        
